In [ ]:
# Global parameters

In [ ]:
folds = 5
seed = 1111
scoring = "neg_mean_absolute_error"

In [ ]:
# Common functionalities

In [ ]:
## Benchmark search algorithm

In [ ]:
def apply_benchmark_model_tuning_and fit(X_train, y_train, parameter_grid, seed, scoring, folds):
    benchmark_neural_network = MLPClassifer(random_state=seed)
    grid_search = GridSearchCV(estimator=benchmark_neural_network, param_grid=parameter_grid, scoring=scoring, cv=folds)
    return grid_search.fit(X_train, y_train)

In [ ]:
## Grid search algorithm

In [ ]:
def grid_serach_neural_network_and_fit(X_train, y_train, scoring, folds):

In [ ]:
## Local search algorithm

In [ ]:
def local_search_neural_network_and_fit(X_train, y_train, scoring, folds):

In [ ]:
# Absenteeism at work

In [ ]:
## Data loading

In [ ]:
absenteeism_at_work_train = pd.read_csv("../../data/absenteeism-at-work/data.csv", delimiter=";", index_col="ID")

In [ ]:
## Train test split

In [ ]:
## Preprocessing

In [ ]:
## Apply benchmark tuning algorithm

In [ ]:
## Apply grid search alogorithm

In [ ]:
## Apply local search algorithm

In [ ]:
## Test all resulting networks

In [ ]:
# Students' dropout and academic success

In [ ]:
## Data loading

In [ ]:
student_dropout_and_academic_sucess_train = pd.read_csv("../../data/predict-students-dropout-and-academic-success/preprocessed_data.csv", delimiter=";")

In [ ]:
## Train test split

In [ ]:
## Preprocessing

In [ ]:
## Apply benchmark tuning algorithm

In [ ]:
## Apply grid search alogorithm

In [ ]:
## Apply local search algorithm

In [ ]:
## Test all resulting networks

In [ ]:
# Loan

In [ ]:
## Data loading

In [ ]:
loans = pd.read_csv("../../data/kaggle-competitions/loan/loan-10k.lrn.csv", index_col="ID")

In [ ]:
## Train test split

In [ ]:
## Preprocessing

In [ ]:
## Apply benchmark tuning algorithm

In [ ]:
## Apply grid search alogorithm

In [ ]:
## Apply local search algorithm

In [ ]:
## Test all resulting networks